In [355]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import codecs
import time
import random
import sqlite3
from datetime import datetime
import pandas as pd
import os
from df2gspread import gspread2df as g2d
import io
import requests

import base64
import re
import rsa
import json
import binascii
import weibo_credentials
import ast
import weibo_credentials

pd.set_option('display.max_rows', 600)

In [ ]:
### SETTINGS

sqlite_file = 'results.sqlite' # name of sqlite file to read from/write to
new_database = False # erases any existing sqlite file and generates an empty one to write to
verbose = "some" # 'none',some','all'
load_cookies = True # load cookies from disk (can load cookies without having to do fresh_log_in if cookies already exist)
fresh_log_in = False # perform a log in
write_cookies = False # save cookies and overwrite any existing cookies during log in
cookie_file = weibo_credentials.Creds().username + "_cookie.txt" # name of cookie file in case you want to specify

censorship_phrase_utf8 = '根据相关法律法规和政策'
censorship_phrase_decoded = codecs.encode(censorship_phrase_utf8.decode('utf8'), 'unicode_escape')

captcha_phrase_utf8 = '你的行为有些异常'
captcha_phrase_decoded = codecs.encode(captcha_phrase_utf8.decode('utf8'), 'unicode_escape')

no_results_phrase_utf8 = '抱歉，未找到'
no_results_phrase_decoded = codecs.encode(no_results_phrase_utf8.decode('utf8'), 'unicode_escape')

In [6]:
class Userlogin:  
    """
    Logs a user into Weibo and generates a cookie
    Returns a Requests session with cookie object
    Pulls username and password from weibo_credentials.py
    code from https://www.zhihu.com/question/29666539 with minor modifications
    """
    def userlogin(self,username,password,write_cookie=True):  
        session = requests.Session()  
        url_prelogin = 'http://login.sina.com.cn/sso/prelogin.php?entry=weibo&callback=sinaSSOController.preloginCallBack&su=&rsakt=mod&client=ssologin.js(v1.4.5)&_=1364875106625'  
        url_login = 'http://login.sina.com.cn/sso/login.php?client=ssologin.js(v1.4.5)'  
  
        #get servertime,nonce, pubkey,rsakv  
        resp = session.get(url_prelogin)  
        json_data  = re.findall(r'(?<=\().*(?=\))', resp.text)[0]
        data       = json.loads(json_data)  


        servertime = data['servertime']  
        nonce      = data['nonce']  
        pubkey     = data['pubkey']  
        rsakv      = data['rsakv']  
  
        # calculate su  
        su  = base64.b64encode(username.encode(encoding="utf-8"))  
  
        #calculate sp  
        rsaPublickey= int(pubkey,16)  
        key = rsa.PublicKey(rsaPublickey,65537)  
        message = str(servertime) +'\t' + str(nonce) + '\n' + str(password)  
        sp = binascii.b2a_hex(rsa.encrypt(message.encode(encoding="utf-8"),key))  
        postdata = {  
                            'entry': 'weibo',  
                            'gateway': '1',  
                            'from': '',  
                            'savestate': '7',  
                            'userticket': '1',  
                            'ssosimplelogin': '1',  
                            'vsnf': '1',  
                            'vsnval': '',  
                            'su': su,  
                            'service': 'miniblog',  
                            'servertime': servertime,  
                            'nonce': nonce,  
                            'pwencode': 'rsa2',  
                            'sp': sp,  
                            'encoding': 'UTF-8',  
                            'url': 'http://weibo.com/ajaxlogin.php?framelogin=1&callback=parent.sinaSSOController.feedBackUrlCallBack',  
                            'returntype': 'META',  
                            'rsakv' : rsakv,  
                            }  
        resp = session.post(url_login,data=postdata)  
        # print resp.headers 
        #print(resp.content)
        login_url = re.findall(r'http://weibo.*&retcode=0',resp.text)  
        #print(login_url)
        respo = session.get(login_url[0])  
        uid = re.findall('"uniqueid":"(\d+)",',respo.text)[0]  
        url = "http://weibo.com/u/"+uid  
        respo = session.get(url)
        if write_cookie:
            cookie_dict = session.cookies.get_dict()
            with open(username + "_cookie.txt", 'w') as f:
                f.write(cookie)
        return session
    
if fresh_log_in:
    session = Userlogin().userlogin(weibo_credentials.Creds().username,weibo_credentials.Creds().password)

if load_cookies:
    with open(cookie_file, 'r') as f:
        cookie = ast.literal_eval(f.read())
else:
    cookie = None

In [92]:
def verify_cookies_work(cookie=cookie,return_full_response=False):
    """
    Returns True if cookies return profile indicator
    If no cookie or bad cookie is passed, you get a generic login page which doesn't have the indicator
    """
    if return_full_response:
        r = requests.get('http://s.weibo.com/weibo/%25E9%25AB%2598%25E8%2587%25AA%25E8%2581%2594&Refer=index',cookies=cookie).content
        return r
    r = requests.get('http://level.account.weibo.com/level/mylevel?from=profile1',cookies=cookie).text
    if "W_face_radius" in r:
        return True
    else:
        return False

In [295]:
def has_censorship(keyword,cookies=None):
    """
    Function which actually looks up whether a search for the given keyword returns text
    which is displayed during censorship.
    Can handle unicode and strings
    Currently no CAPTCHA handling, though it is detected
    Returns string of 'censored','no_results','reset',or 'has_results'
    ('has_results' is actually not a garuantee; it's merely a lack of other censorship indicators)
    """
    if isinstance(keyword, str):
        url = 'http://s.weibo.com/weibo/%s&Refer=index' % keyword
    elif isinstance(keyword, unicode):
        url = ('http://s.weibo.com/weibo/%s&Refer=index' % keyword).encode('utf-8')    
    
    try:
        r = requests.get(url,cookies=cookie).text
        i = 1
        while True:
            if captcha_phrase_decoded not in r:
                break
            else:
                print "CAPTCHA", keyword
                time.sleep(300*i)
                i+=1
    except IOError:
        wait_seconds = random.randint(90, 100)
        print "connection reset, waiting %s" % wait_seconds
        time.sleep(wait_seconds)
        return "reset"

    num_results = re.findall(r'search_rese clearfix\\">\\n <span>\\u627e\\u5230(\d*)',r)
    if num_results:
        num_results = int(num_results[0])
    else:
        None
    
    if censorship_phrase_decoded in r:
        return ("censored",None)
    elif no_results_phrase_decoded in r:
        return ("no_results",None)
    else:
        return ("has_results",num_results)

In [79]:
def create_table(sqlite_file):
    """
    Generating a sqlite file for storing results
    Multi-index primary key on id, date, and source
    Set new_database to True in order to remove any existing file
    """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute('CREATE TABLE results (id int, date date, datetime datetime, keyword string, censored bool, no_results bool, reset bool, result string, source string, num_results int, notes string, PRIMARY KEY(id,date,source))')
    conn.commit()
    conn.close()
    
if new_database and os.path.isfile(sqlite_file):
    os.remove(sqlite_file)
if not os.path.isfile(sqlite_file):
    create_table(sqlite_file)

In [320]:
def insert_into_table(record_id,keyword,result,source,notes=None,num_results=None):
    """
    Writing the results to the sqlite database file
    """
    conn = sqlite3.connect(sqlite_file)
    conn.text_factory = str
    c = conn.cursor()
    
    dt = datetime.now()
    d = dt.date()
    
    if result is "censored":
        censored = True
    else:
        censored = False
        
    if result is "no_results":
        no_results = True
    else:
        no_results = False
        
    if result is "reset":
        reset = True
    else:
        reset = False

    query = """INSERT INTO results (id, date, datetime, keyword, censored, no_results, reset, result, source, num_results, notes) VALUES (?,?,?,?,?,?,?,?,?,?,?);"""
    if isinstance(notes, list):
        notes = str(notes)
    c.execute(query,(record_id, d, dt, keyword, censored, no_results, reset, result, source, num_results, notes))

    conn.commit()
    conn.close()
    
def sqlite_to_df(sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    df = pd.read_sql_query("select * from results where source!='_meta_';", conn)
    return df

In [354]:
def get_keywords_from_source(location,keyword_col_name,source_name,lxb_categories=None):
    """
    Generating the keyword lists to search on
    """
    test_keywords = pd.DataFrame()
    if '.csv' in location:
        s=requests.get(location).content
        test_df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    elif "recommendation" in location:
        pass
    else:
        test_df = g2d.download(location,wks_name='Sheet2',col_names=True)
    if lxb_categories:
        mask = test_df.category.isin(lxb_categories)
        test_df = test_df[mask]
    if '.csv' in location:
        test_keywords['category'] = test_df.category
    test_keywords['keyword'] = test_df[keyword_col_name]
    test_keywords['source'] = source_name
    test_keywords['notes'] = None
    return test_keywords

In [332]:
def run(test_keywords,verbose='some',insert=True,return_df=False,sleep=True):
    """
    Iterating through the keyword list and testing one at a time
    Handles when script or connection breaks; will pick up where it left off
    Set return_df to append each new result to a df in memory, which is returned at end of function
    verbose = 'some','all',or 'none'(technically anything besides 'some' or 'all' will not show anything)
    """
    count=0
    if return_df:
        results_df = pd.DataFrame()
    
    for r in test_keywords.itertuples():
        if insert and r.Index < len(sqlite_to_df(sqlite_file)):
            continue
        result,num_results = has_censorship(r.keyword)
        if verbose=="all":
            print r.Index,r.keyword, result
        elif verbose=="some" and (count%10==0 or count==0):
            print r.Index,r.keyword, result
        if insert:
            insert_into_table(len(sqlite_to_df(sqlite_file)),r.keyword,result,r.source,num_results,r.notes)
        if return_df:
            results_df = pd.concat([results_df,
                                    pd.DataFrame([{"date":datetime.now().date(),
                                                   "datetime":datetime.now(),
                                                   "keyword":r.keyword,
                                                   "result":result,
                                                   "source":r.source,
                                                   'num_results':num_results
                                                 }])
                                   ])
        count+=1
        if sleep:
            time.sleep(random.randint(13, 16))
    if insert:
        insert_into_table(int(test_keywords.index.max())+1,None,"finished","_meta_")
    if return_df:
        return results_df

In [38]:
sample_keywords = pd.DataFrame(
    [{'keyword':'hello','Index':0,'source':'test'},
     {'keyword':'lxb','Index':1,'source':'test'},
     {'keyword':u'习胞子','Index':2,'source':'unicode'},
     {'keyword':'自由亚洲电台','Index':3,'source':'should reset'},
     {'keyword':'刘晓波','Index':4,'source':'string'},
     {'keyword':'dhfjkdashfjkasdhfsadsf87sadfhjfasdnf'}])

run(sample_keywords,verbose='none',insert=False,return_df=True)

connection reset, waiting 90


,date,datetime,keyword,result,source
0,2017-07-14,2017-07-14 06:12:48.502143,hello,has_results,test
0,2017-07-14,2017-07-14 06:13:05.141758,lxb,censored,test
0,2017-07-14,2017-07-14 06:13:23.329775,习胞子,no_results,unicode
0,2017-07-14,2017-07-14 06:15:09.901741,自由亚洲电台,reset,should reset
0,2017-07-14,2017-07-14 06:15:31.526905,刘晓波,censored,string
0,2017-07-14,2017-07-14 06:15:52.814367,dhfjkdashfjkasdhfsadsf87sadfhjfasdnf,no_results,NaN


In [74]:
lxb_categories = ["Cultural Revolution","Dissident","charter 08","human rights","Charter 08","Dissident / Activist","Detention","Human Rights","Tiananmen Square (June 4 1989)"]
df1 = get_keywords_from_source("wechat_keywords","Keyword","wechat spreadsheet")
df2 = get_keywords_from_source("https://raw.githubusercontent.com/citizenlab/chat-censorship/master/livestream/livestream_keywords_05_15_09_16.csv",'word','livestream_05_15_09_16',lxb_categories=lxb_categories)
df3 = get_keywords_from_source("https://raw.githubusercontent.com/citizenlab/chat-censorship/master/TOM-Skype--Sina-UC/blocked-words.csv","word","tom-skype-sina",lxb_categories=lxb_categories)
test_keywords = pd.concat([df1,df2,df3]).reset_index(drop=True)

In [77]:
print len(test_keywords)
test_keywords.head()

1612


,keyword,source
0,退出中共,wechat spreadsheet
1,中革中央,wechat spreadsheet
2,共产党下台,wechat spreadsheet
3,突破中共,wechat spreadsheet
4,废除中共,wechat spreadsheet


In [ ]:
run(test_keywords)

In [263]:
lxb_keywords = ['lxiaob','lxb','刘晓波','劉曉波','lx','liux','刘晓']
suggested_keywords = {}
for l in lxb_keywords:
    r = requests.get("http://s.weibo.com/ajax/suggestion?where=gs_weibo&key=%s&count=100" % l,cookies=cookie)
    for item in json.loads(r.text)['data']:
        suggested_keywords[item['key']] = (item['count'],l)

In [274]:
suggested_keywords_df = pd.DataFrame(suggested_keywords.items(),columns=['keyword','info'])
suggested_keywords_df['notes'] = suggested_keywords_df.apply(lambda row:row['info'][0],axis=1)
suggested_keywords_df['source'] = suggested_keywords_df.apply(lambda row:'auto_suggest_search_' + row['info'][1],axis=1)
suggested_keywords_df.drop('info',axis=1,inplace=True)
suggested_keywords_df.index = np.arange(len(sqlite_to_df(sqlite_file)), len(suggested_keywords_df) + len(sqlite_to_df(sqlite_file)))

In [ ]:
run(suggested_keywords_df,verbose='none')

In [322]:
lxb_keywords = ['liuxiaobo','刘小','肝癌','liuxb']
suggested_keywords_2 = {}
for l in lxb_keywords:
    r = requests.get("http://s.weibo.com/ajax/suggestion?where=gs_weibo&key=%s&count=100" % l,cookies=cookie)
    for item in json.loads(r.text)['data']:
        suggested_keywords_2[item['key']] = (item['count'],l)

In [323]:
suggested_keywords2_df = pd.DataFrame(suggested_keywords_2.items(),columns=['keyword','info'])
suggested_keywords2_df['notes'] = suggested_keywords2_df.apply(lambda row:row['info'][0],axis=1)
suggested_keywords2_df['source'] = suggested_keywords2_df.apply(lambda row:'auto_suggest_search_' + row['info'][1],axis=1)
suggested_keywords2_df.drop('info',axis=1,inplace=True)
suggested_keywords2_df.index = np.arange(len(sqlite_to_df(sqlite_file)), len(suggested_keywords2_df) + len(sqlite_to_df(sqlite_file)))

In [324]:
suggested_keywords2_df

,keyword,notes,source
1748,刘小鹿的猫,406,auto_suggest_search_刘小
1749,刘小逗,743,auto_suggest_search_刘小
1750,刘晓北 王梓,428,auto_suggest_search_liuxb
1751,肝癌 去世,862,auto_suggest_search_肝癌
1752,刘小枣的衣橱,690,auto_suggest_search_刘小
1753,刘晓北,978,auto_suggest_search_liuxb
1754,刘星表情包,2271,auto_suggest_search_liuxb
1755,刘小北,115,auto_suggest_search_liuxb
1756,刘小小元,798,auto_suggest_search_刘小
1757,刘小东,537,auto_suggest_search_刘小


In [326]:
run(suggested_keywords2_df)

1805 刘小光出轨女粉丝 has_results


In [342]:
lxb_categories = ["Cultural Revolution","Dissident","charter 08","human rights","Charter 08","Dissident / Activist","Detention","Human Rights","Tiananmen Square (June 4 1989)"]
test_keywords_2 = get_keywords_from_source("wechat_keywords","Keyword","wechat spreadsheet")
test_keywords_2.index = np.arange(len(sqlite_to_df(sqlite_file)), len(test_keywords_2) + len(sqlite_to_df(sqlite_file)))

In [344]:
run(test_keywords_2)

1812 晓波+蔡英文 censored
1822 晓波+逝世 censored
1832 天安门+四君子 censored
1842 习近平+镇压+异议人士 censored


In [345]:
results_df = sqlite_to_df(sqlite_file)
results_df

,id,date,datetime,keyword,censored,no_results,reset,result,source,num_results,notes
0,0,2017-07-14,2017-07-14 06:40:51.236298,退出中共,1,0,0,censored,wechat spreadsheet,NaN,None
1,1,2017-07-14,2017-07-14 06:41:09.308436,中革中央,0,0,0,has_results,wechat spreadsheet,NaN,None
2,2,2017-07-14,2017-07-14 06:41:26.242722,共产党下台,0,0,0,has_results,wechat spreadsheet,NaN,None
3,3,2017-07-14,2017-07-14 06:41:47.062671,突破中共,0,0,0,has_results,wechat spreadsheet,NaN,None
4,4,2017-07-14,2017-07-14 06:42:06.424144,废除中共,0,0,0,has_results,wechat spreadsheet,NaN,None
5,5,2017-07-14,2017-07-14 06:42:27.316125,打倒中共,1,0,0,censored,wechat spreadsheet,NaN,None
6,6,2017-07-14,2017-07-14 06:42:45.258756,打倒共产党,1,0,0,censored,wechat spreadsheet,NaN,None
7,7,2017-07-14,2017-07-14 06:43:04.460774,推翻中共,1,0,0,censored,wechat spreadsheet,NaN,None
8,8,2017-07-14,2017-07-14 06:43:23.154911,灭中共,0,0,0,has_results,wechat spreadsheet,NaN,None
9,9,2017-07-14,2017-07-14 06:43:44.041456,胡温暴政,1,0,0,censored,wechat spreadsheet,NaN,None


todo: 
     include run number, change primary key,

In [346]:
results_df.groupby(['result','source']).size().to_frame()

0
result      source                            
censored    livestream_05_15_09_16           9
            tom-skype-sina                 114
            wechat spreadsheet             466
has_results auto_suggest_search_liux        35
            auto_suggest_search_liuxb       14
            auto_suggest_search_liuxiaobo    2
            auto_suggest_search_lx          31
            auto_suggest_search_lxb         31
            auto_suggest_search_lxiaob      14
            auto_suggest_search_刘小          42
            auto_suggest_search_刘晓          24
            auto_suggest_search_肝癌           6
            livestream_05_15_09_16          31
            tom-skype-sina                 353
            wechat spreadsheet             149
no_results  auto_suggest_search_liux         1
            livestream_05_15_09_16           2
            tom-skype-sina                  48
            wechat spreadsheet             470
reset       wechat spreadsheet               4

In [329]:
results_df.groupby(['source']).size().to_frame()

,0
source,
auto_suggest_search_liux,36
auto_suggest_search_liuxb,14
auto_suggest_search_liuxiaobo,2
auto_suggest_search_lx,31
auto_suggest_search_lxb,31
auto_suggest_search_lxiaob,14
auto_suggest_search_刘小,42
auto_suggest_search_刘晓,24
auto_suggest_search_肝癌,6


In [363]:
results_df_less = results_df.query("source!='wechat_keywords' | (source=='wechat_keywords' and ( \
                 notes=='Release of Liu Xiaob into Hospitical'))")

In [415]:
temp_df = results_df_less[~((results_df_less['keyword'].str.contains(u"劉曉波|刘晓波")) & (results_df_less['keyword'].str.len()>3))]

In [416]:
from df2gspread import df2gspread as d2g
d2g.upload(temp_df,"wechat_keywords",wks_name='results2',df_size=True)

<Worksheet 'results2' id:oqrhd07>

In [417]:
results_df_less[~((results_df_less['keyword'].str.contains(u"劉曉波|刘晓波")) & (results_df_less['keyword'].str.len()>3))].groupby('keyword').size().to_frame("count").sort_values('count')

,count
keyword,
3.14,1
民主运动+烛光晚会+六四,1
民主歌声献中华,1
民主抗爭+有錢人壟斷+反對中共,1
民主抗争+有钱人垄断+反对中共,1
民主女神,1
毛澤東發動+暴力的文革+習近平,1
毛泽东发动+暴力的文革+习近平,1
毛泽东+大权独揽+屠杀,1
